# SVM for classification
This implements SVM to solve binary classification problems.  
We minimize hinge loss function with penalty term:  
$$min \sum_{n=1}^Nmax(0,1-yi*(w^T*xi))+\rho*(w^T*w)$$

#### Notation
w = [w1, w2, ..., b]  
xi = [\\(x^{(1)}\\), \\(x^{(2)}\\), ...]  
yi is 0 or 1  
Training set is {(X,Y)} where X = [x1,x2, ..., 1] and Y = [y1, y2 ...].
## Stochatic subgradient descent for SVM